In [ ]:
import os
import pickle
import sys

import matplotlib.pyplot as plt
import numpy as np
from pprint import pprint
from hydra import initialize, compose
from glob import glob
import multiprocessing

# set paths to various directories based on the machine this code is being executed on
with initialize(version_base=None, config_path='../../config'):
    config = compose(config_name='config.yaml')  # overrides=['machine=uzay']

array_dir, data_dir, figure_dir, pickle_dir, repo_dir  = config.machine.array_dir, config.machine.data_dir, config.machine.figure_dir, config.machine.pickle_dir, config.machine.repo_dir

# enable use of local modules
if repo_dir not in sys.path:
    sys.path.append(repo_dir)

# set matplotlib style
plt.style.use(f'{repo_dir}/mejiro/mplstyle/science.mplstyle')

from mejiro.plots import diagnostic_plot, plot
from mejiro.utils import util
from mejiro.helpers import pyhalo

In [ ]:
array_dir = os.path.join(array_dir, 'scenes_across_detectors')
util.create_directory_if_not_exists(array_dir)

In [ ]:
offset_list = [(0, 0), (-1, -1), (-1, 1), (1, -1), (1, 1)]

grid_oversample = 1
num_samples = 1000

In [ ]:
# use test lens
lens = test_physical_lens.TestPhysicalLens()

# add CDM subhalos; NB same subhalo population for all
with open(os.path.join(pickle_dir, 'cdm_subhalos_for_sample_skypy_lens'), 'rb') as results_file:
    realizationCDM = pickle.load(results_file)
lens.add_subhalos(*pyhalo.realization_to_lensing_quantities(realizationCDM))

model = lens.get_array(num_pix=num_pix * grid_oversample, side=side + buffer)

In [ ]:
# build Pandeia input
calc, _ = pandeia_input.build_pandeia_calc(csv=csv,
                                        array=model, 
                                        lens=lens, 
                                        band='f106',
                                        num_samples=num_samples)

# do Pandeia calculation        
image, execution_time = pandeia_input.get_pandeia_image(calc)

# save detector image
np.save(os.path.join(array_dir, f'test_physical_lens_image_{num_samples}'), image)

In [ ]:
cpu_count = multiprocessing.cpu_count()
generator = util.batch_list(lens_list, cpu_count)
batches = list(generator)

for batch in tqdm(batches):
    pool = Pool(processes=cpu_count) 
    output = []
    for each in pool.map(generate.main, batch):
        output.append(each)

In [ ]:
image_list, execution_times, num_point_sources = [], [], []

for tuple in output:
    (image, execution_time, num_ps) = tuple
    image_list.append(image)
    execution_times.append(execution_time)
    num_point_sources.append(num_ps)

In [ ]:
image_list = [i for i in image_list if i is not None]

plot.plot_grid(image_list, side=16)

In [ ]:
for image in image_list:
    np.save(os.path.join(array_dir, f'skypy_output_{str(i).zfill(5)}'), image)